In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/merged-traintest/train_merged.csv')
test=pd.read_csv('/kaggle/input/merged-traintest/test_merged.csv')


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

# set target
y = train['score']
#y=np.array(y)


#use keras to create a Tokenizer object
tokenizer = text.Tokenizer(num_words=20000)  # limit to the num_words most important ones
tokenizer.fit_on_texts(list(train['text']))

tokenized_texts_train = tokenizer.texts_to_sequences(train['text'])
tokenized_texts_test = tokenizer.texts_to_sequences(test['text'])

X_train = sequence.pad_sequences(tokenized_texts_train, maxlen=100)
X_test = sequence.pad_sequences(tokenized_texts_test, maxlen=100)

In [ ]:
for i in range(0,len(y)):
    if y[i]==0.50:
        y[i]=0
    elif y[i]==0.25:
        y[i]=1
    elif y[i]==0.00:
        y[i]=2
    elif y[i]==0.75:
        y[i]=3
    elif y[i]==1.00:
        y[i]=4

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)

# create my NN model
model = Sequential()

embedding_size = 128
model.add(Embedding(20000, embedding_size,input_length=100))
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=20)

In [ ]:
test_scores=model.predict(X_test)

In [ ]:
test_scores

In [ ]:
final_scores=[]
for i in range(0,len(test_scores)):
    res=np.argmax(test_scores[i])
    final_scores.append(res)

In [ ]:
final_scores

In [ ]:
for i in range(0,len(final_scores)):
    if final_scores[i]==0:
        final_scores[i]=0.50
    elif final_scores[i]==1:
        final_scores[i]=0.25
    elif final_scores[i]==2:
        final_scores[i]=0
    elif final_scores[i]==3:
        final_scores[i]=0.75
    elif final_scores[i]==4:
        final_scores[i]=1

In [ ]:
final_scores

In [ ]:
sub=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv')

In [ ]:
sub_csv = []
sub_csv=pd.DataFrame(sub_csv)
sub_csv['id']=sub['id']
sub_csv['score']=final_scores
sub_csv.head()

In [ ]:
sub_csv.to_csv('submission.csv',index=False)